In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import sys
print sys.getdefaultencoding()

import codecs
import data_utils
import seq2seq_model
import re

import jieba
from snownlp import SnowNLP

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
from data_utils import *

ascii
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2811332191905368984
]


In [ ]:
# NOTES:

# chinese comma 
# numbers 
# @ symbol 


# TO DO:
# 1. merge or agumented vocabulary library 
# 2. beam search on decoder, diversified beam search
# 3. sampled softmax
# 4. word embedding 
# 5. phrase based decoder 
# 6. intention
# 7. style transfer: speaker style, 


# https://blog.kovalevskyi.com/rnn-based-chatbot-for-6-hours-b847d2d92c43
# http://www.franksworld.com/2017/04/05/how-to-make-a-deep-learning-chatbot-with-keras-and-python/

In [ ]:
#  reference: https://github.com/candlewill/Dialog_Corpus

In [7]:
# functions for data splitting 

def remove_symbol_at( sentence ):
    withinAt=False
    res = []
    for i in sentence:
        if i=='@':
            if withinAt == True:
                withinAt = False
            else:
                withinAt = True
            
        elif withinAt == False:
            res.append(i)
            withinAt = False
            
    return res

def remove_number(sentence):
    _DIGIT_RE = re.compile(br"\d")
    res = []
    
    for i in sentence:
        if re.sub(_DIGIT_RE, b"0", i) == i:
            res.append(i)
    return res 

def normalize_number(sentence):
    _DIGIT_RE = re.compile(br"\d")
    res = []
    
    for i in sentence:
        res.append(re.sub(_DIGIT_RE, b"0", i))
    return res 

def print_out_cn( sentence ):
    for i in sentence:
        print i.encode('utf-8')

def file_encoder_decoder_pair( enc_path, dec_path, text_line, idx):
    
    tmp = text_line.split('\t')
#     print len(tmp)
    
    if len(tmp)<2:
        return False
    
    enc_str = remove_symbol_at( tmp[idx].strip() )
    dec_str = remove_symbol_at( tmp[idx+1].strip() )
    
    enc_str = remove_number( enc_str )
    dec_str = remove_number( dec_str )
    
    with open(enc_path, "a") as enc_file:
        
        for j in enc_str:
            enc_file.write("%s" %j.encode('utf-8'))
            
        enc_file.write("\n")
        
    with open(dec_path, "a") as dec_file:
        
        for j in dec_str:
            dec_file.write("%s" %j.encode('utf-8'))
            
        dec_file.write("\n")
    
    return True

def file_ini( path ):
    with open(path, "w") as file_handler:
        file_handler.close()
        
# TO DO: 
def tokenize_symbol_at( sentence ):
    return 0       

In [14]:
# read and split the dataset 
import codecs

# source data format:
# a set of pairs of < question \t answer > 
# 

src_file_path = '../../dataset/nlp/huarui/huarui.weibo.all.star.QA.txt'

f1 = codecs.open(src_file_path, "r", "utf-8")

num_lines = sum(1 for line in open(src_file_path))
num_train = int(0.8*num_lines)

file_ini( "../../dataset/nlp/data_cn/train.cn.enc" )
file_ini( "../../dataset/nlp/data_cn/train.cn.dec" )
file_ini( "../../dataset/nlp/data_cn/test.cn.enc" )
file_ini( "../../dataset/nlp/data_cn/test.cn.dec" )

# training data
for i in range(num_train):
    text = f1.readline()
    if not file_encoder_decoder_pair( "../../dataset/nlp/data_cn/train.cn.enc",\
                                     '../../dataset/nlp/data_cn/train.cn.dec', text, 1):
        print i, text

# testing data
cnt = i
text = f1.readline()
while text:
    cnt+=1
    if not file_encoder_decoder_pair( '../../dataset/nlp/data_cn/test.cn.enc',\
                                      '../../dataset/nlp/data_cn/test.cn.dec', text, 1):
        print cnt, text
    text = f1.readline()

print 'DONE!'

1113 来大济南啦@！映后见面会主创们和观众聊的很开心，冰冰还用家乡话跟朋友们问好！下一站沈阳，明天见妆发：@张浩然Rstudio@服装：@ChristopherBu@

4429  
4430  
4431 但得而不待，时不再来生命无关途中际遇，兀自向前 
4432  
4433 终于明白，幸福不在终点，幸福就是此生此路 
4434 请珍视每一寸光阴，珍视每一个共同渡过的人相信同伴就是一种珍贵AlfredDSouza

DONE!


In [2]:
try:
    from ConfigParser import SafeConfigParser
except:
    from configparser import SafeConfigParser
    
def get_config(config_file='seq2seq.ini'):
    parser = SafeConfigParser()
    parser.read(config_file)
    # get the ints, floats and strings
    _conf_ints = [ (key, int(value)) for key,value in parser.items('ints') ]
    _conf_floats = [ (key, float(value)) for key,value in parser.items('floats') ]
    _conf_strings = [ (key, str(value)) for key,value in parser.items('strings') ]
    return dict(_conf_ints + _conf_floats + _conf_strings)

gConfig = get_config()

gConfig

{'batch_size': 64,
 'dec_vocab_size': 20000,
 'enc_vocab_size': 20000,
 'layer_size': 256,
 'learning_rate': 0.5,
 'learning_rate_decay_factor': 0.99,
 'max_gradient_norm': 5.0,
 'max_train_data_size': 0,
 'mode': 'local_test',
 'num_layers': 3,
 'steps_per_checkpoint': 500,
 'test_dec': '../../dataset/nlp/data_cn/test.cn.dec',
 'test_enc': '../../dataset/nlp/data_cn/test.cn.enc',
 'train_dec': '../../dataset/nlp/data_cn/train.cn.dec',
 'train_enc': '../../dataset/nlp/data_cn/train.cn.enc',
 'working_directory': 'working_dir_cn/'}

In [ ]:
enc_train, dec_train, enc_dev, dec_dev, _, _ = \
data_utils.prepare_custom_data(gConfig['working_directory'],gConfig['train_enc'],gConfig['train_dec'],\
                    gConfig['test_enc'],gConfig['test_dec'],gConfig['enc_vocab_size'],gConfig['dec_vocab_size'] )


In [ ]:
# statistics on encoder-decoder pairs

file_paths = [ "../../dataset/nlp/data_cn/train.cn.enc.ids20000",\
               "../../dataset/nlp/data_cn/train.cn.dec.ids20000",\
               "../../dataset/nlp/data_cn/test.cn.enc.ids20000",\
               "../../dataset/nlp/data_cn/test.cn.dec.ids20000" ]

for i in file_paths:
    
    f1 = codecs.open(i, "r")
        
    line = f1.readline()
    line_len = []
    
    while line:
        line_len.append( len(line.split(' ')) )
        line = f1.readline()
        
    print i,"  ", \
          "Mean:", sum(line_len)*1.0/len(line_len),\
          "Max:", max(line_len),\
          "Min:", min(line_len)
    

In [ ]:
# test on snownlp and jieba
from snownlp import *

sentence = u"   从过去获取记忆的灵感，创造121明天未知12123的潮流，212。@范冰冰@@范冰冰工作室@ 【范冰冰 】@路易威登@： ..."

sentence = u"海报来也！大家一起数星星→       梁家辉 祖峰    众神归位！请用一个词点评本片阵容，\
评论里最有才的那位，片方将免费送你去上影节“全明星封神之夜”哟！"

sentence = u'???? ????<br/>奥汀子集齐，月刊召唤成功！ 、 、 、 、 、 、 及 携手呈现双封面！明天起将每天揭晓位主创的网络专属单人封面，\
更有可能获得独属你家偶像的签名海报套装，敬请锁定陆续开启的购买链接 ...'

sentence = u'期待值up up but fdafa！ 大哥英雄力MAX 星约翰尼•诺克斯维尔,在澳大利亚总理夫人Lucy Turnbull女士的见证下\
先转藏再看by网络 http:t.cn/Rq0Erd'

sentence = u'冰冰身着马海毛121运动双排扣大衣132323'

sentence = u'#分手大师#好演员'
# snownlp
s = SnowNLP(sentence)
_DIGIT_RE = re.compile(br"\d")

# split on words
for i in s.words:
    
    word = re.sub(_DIGIT_RE, b"0", i)
    if word == i:
        print word.encode('utf-8')
    else:
        print word.encode('utf-8')
        
        
# # jieba
# seg_list = jieba.cut(sentence)

# tokens = []
# for i in seg_list:
#     print i
#     tokens.append(i)
